In [ ]:
from __future__ import division
import pandas as pd
import numpy as np 
import matplotlib.pylab as plt
from code.organize import *
from code.roughEDA import *
from code.survey_processor_full import *
from code.model_vis import *

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix,roc_curve
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA

from treeinterpreter import treeinterpreter as ti
import matplotlib.cm as cm
import cPickle as pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

To further classify employed and unemployed women, 

In [ ]:
## 4. Clustering Feature Contribution Matrix

* pca
* kmeans on principle components

### PCA

fc_f_pca = fc_df.drop(['employed','correct'],axis =1)       #only pass feature columns to pca

pca = PCA(n_components= 10)
pca.fit(fc_f_pca.values)

np.cumsum(pca.explained_variance_ratio_),pca.explained_variance_ratio_

plt.plot(range(1,11),pca.explained_variance_ratio_)

** Three principle components will be fine **

pca = PCA(n_components=3)
fc_pca = pca.fit_transform(fc_f_pca.values)

### Kmeans

To determine which k for kmeans. I wrote a function to check kmeans cluster performance. 

1. Take a range of Ks and make a for loop

2. In each loop：
  * Generate k clusters using kmeans. concatinate cluster id to 'employed' and 'correct' columns, as a new data frame.
  * Group data frame by cluster id, calcuate employed rate and correct rate by taking means
  * If the employed rate is higher than 0.7 or lower than 0.3, then we can say this cluster successfull group employed or unemployed population, so this cluster yields pure population.
  * If the correct rate is higher than 0.7, we can say this cluter has high accuracy.
  * The clusters with pure population and meantime with high accuracy are ones we want. 
  * Therefore, the percentage of good clusters is calculated.
  * Print employed and correct rate of each good cluster, as well as the good cluster ratio.
  
3. plot total number of k against good cluster ratio

km_emp_mean(fc_pca,10,fc_df.employed,fc_df.correct)

plot_silhouette(fc_pca,10)

cluster_plot(fc_pca,6)

According to the results, all the Kmeans cluster showed high good cluster ratios. When the number of cluster for kmeans is six, it has 100% good cluster ratio, 3 employed, 3 unemployed. Moreover, it exhibited very high silhouette score. Thus, n_cluster will be set to 6.

km = KMeans(n_clusters=6,random_state=0)
km.fit(fc_pca)
fc_df['cluster'] = km.labels_

## Next step:

1. Pickle dfc[slct] and fc_df

2. Analyse average feature contribution in each cluster